In [1]:
#2
import sqlite3

In [2]:
def __init__db():
    connection = sqlite3.connect("LBMS.db")
    cursor =  connection.cursor()
    print("Database connection established successfully")
    create_table(cursor) 
    return (connection,cursor)

In [3]:
def term_db(connection):
    connection.close()
    print("Database connection de-established successfully")


In [4]:
def create_table(cursor):
    sql_command = '''CREATE TABLE if not exists books (
        title text,
        author text,
        year decimal,
        ISBN INTEGER,
        unique (title, author, year, ISBN)
    )'''
    cursor.execute(sql_command)  

In [5]:
def read_table(cursor):

    sql_command = "SELECT * FROM books"
    cursor.execute(sql_command)  
    data = cursor.fetchall()
    return(data)


def add_to_table(cursor, title, author, year, isbn):
    
    sql_command = f"INSERT INTO books VALUES ('{title}', '{author}', {year}, {isbn});"
    try:
        cursor.execute(sql_command)
        return(0, "success")
    except sqlite3.IntegrityError as e:
        return(111, e)
    except sqlite3.OperationalError as e:
        return(112, e)

In [6]:
def remove_from_table(cursor, title, author, year, isbn):
    try:
        sql_command = f'''DELETE FROM books WHERE title="{title}";'''
        cursor.execute(sql_command)
        sql_command = f'''DELETE FROM books WHERE author="{author}";'''
        cursor.execute(sql_command)
        sql_command = f'''DELETE FROM books WHERE year="{year}";'''
        cursor.execute(sql_command)
        sql_command = f'''DELETE FROM books WHERE isbn="{isbn}";'''
        cursor.execute(sql_command)
    except:
        return(-1)

In [7]:
def search_from_table(cursor, title, author, year, isbn):
   
    try:
        cursor.execute("SELECT * FROM books WHERE title=?", (title,))
        searched_titles = cursor.fetchall()
        cursor.execute("SELECT * FROM books WHERE author=?", (author,))
        searched_authors = cursor.fetchall()
        cursor.execute("SELECT * FROM books WHERE year=?", (year,))
        searched_years = cursor.fetchall()
        cursor.execute("SELECT * FROM books WHERE ISBN=?", (isbn,))
        searched_isbns = cursor.fetchall()
        return(searched_titles + searched_authors + searched_years + searched_isbns)
    except:
        return(-1)


def save_changes(connection):
    
    connection.commit()

In [8]:
def list_all_books(listArea):
    connection, cursor = init_db()
    print("listing books")
    books = read_table(cursor)
    listArea.delete('0','end')
    for i in range(len(books)):
        listArea.insert(i,f"{books[i][0]}, {books[i][1]}, {books[i][2]}, {books[i][3]}")
    term_db(connection)

In [9]:
def search_book(listArea, title, author, year, isbn):
    connection, cursor = init_db()
    print("search book")
    title = title.get()
    author = author.get()
    year = year.get()
    isbn = isbn.get()
    res = search_from_table(cursor, title, author, year, isbn)
    if res==-1:
        messagebox.showerror("Error!", "something went wrong!")
    if res!=-1:
        listArea.delete('0','end')
        for i in range(len(res)):
            listArea.insert(i,res[i])  
    term_db(connection)

In [10]:
def add_book(title, author, year, isbn):
    connection, cursor = init_db()
    print("adding book")
    title = title.get()
    author = author.get()
    year = year.get()
    isbn = isbn.get()
    code, error = add_to_table(cursor, title, author, year, isbn)
    if code==111:
        messagebox.showerror("Error!", f"Book Already Exists!\n\nerror code: {code}\nerror:{error}")
    if code==112:
        messagebox.showerror("Error!", f"Please, fill in the input fields appropriately.\n\ntitle: string, author: string, year: integer, year: integer.\n\nerror code: {code}\nerror:{error}")
    save_changes(connection)
    term_db(connection)


In [11]:
def issue_book(title, author, year, isbn):
    connection, cursor = init_db()
    print("issue book")
    messagebox.showwarning("Undefined", "this function is not yet defined!")
    term_db(connection)

In [12]:
def delete_book(title, author, year, isbn):
    connection, cursor = init_db()
    print("delete book")
    title = title.get()
    author = author.get()
    year = year.get()
    isbn = isbn.get()
    code = remove_from_table(cursor, title, author, year, isbn)
    if code==-1:
        messagebox.showerror("Error!", "something went wrong!")
    messagebox.showinfo("Deleted!", f"Deleted the book: {title}, {author}, {year}, {isbn}")        
    save_changes(connection)
    term_db(connection)


In [18]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

def gui():
    # 1. root
    root = Tk()
    root.title('Library Book Management System')

    # 2. widgets
    title = StringVar()
    author = StringVar()
    year = StringVar()
    isbn = StringVar()

    title_label = Label(root, text="title:", padx=10)
    author_label = Label(root, text="author:", padx=10)
    year_label = Label(root, text="year:", padx=10)
    isbn_label = Label(root, text="isbn:", padx=10)

    title_entry = Entry(root, textvariable=title)
    author_entry = Entry(root, textvariable=author)
    year_entry = Entry(root, textvariable=year)
    isbn_entry = Entry(root, textvariable=isbn)


    scrollbar = Scrollbar(root, orient='horizontal')
    listArea = Listbox(root, bg='white', width=100, yscrollcommand=scrollbar.set)

    list_all_btn = Button(root, text="List All", command=lambda:list_all_books(listArea))
    search_btn = Button(root, text="Search", command=lambda:search_book(listArea, title, author, year, isbn))
    add_btn = Button(root, text="Add", command=lambda:add_book(title, author, year, isbn))
    issue_btn = Button(root, text="Issue", command=lambda:issue_book(title, author, year, isbn))
    del_btn = Button(root, text="delete", command=lambda:delete_book(title, author, year, isbn))
    exit_btn = Button(root, text="Exit", command=root.destroy)

    scrollbar.config(command=listArea.yview)

    title_label.grid(row=0, column=0)
    author_label.grid(row=0, column=2)
    year_label.grid(row=0, column=4)
    isbn_label.grid(row=0, column=6)


    title_entry.grid(row=0, column=1)
    author_entry.grid(row=0, column=3)
    year_entry.grid(row=0, column=5)
    isbn_entry.grid(row=0, column=7)

    listArea.grid(row=1, column=0, columnspan=8, sticky="nsew")
    scrollbar.grid(row=1, column=9, sticky="ns")

    list_all_btn.grid(row=2, column=0)
    search_btn.grid(row=2, column=1)
    add_btn.grid(row=2, column=2)
    issue_btn.grid(row=2, column=3)
    del_btn.grid(row=2, column=4)
    exit_btn.grid(row=2, column=7)
    print("GUI loop INITIATING")
    root.mainloop()
    print("GUI loop TERMINATED")
 

gui()

GUI loop INITIATING


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Sakthi\AppData\Local\Temp\ipykernel_11860\3353563413.py", line 30, in <lambda>
    list_all_btn = Button(root, text="List All", command=lambda:list_all_books(listArea))
  File "C:\Users\Sakthi\AppData\Local\Temp\ipykernel_11860\1860388818.py", line 2, in list_all_books
    connection, cursor = init_db()
NameError: name 'init_db' is not defined


GUI loop TERMINATED
